In [ ]:
import tensorflow as tf
import numpy as np

# Constants and settings
learning_rate = 0.001
epochs = 1000
a = 10  # Truncation of the real line for the loss calculation
alpha = 0.1  # Alpha parameter from the equation

# Define a custom layer for the characteristic transformation
class CharacteristicLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(CharacteristicLayer, self).__init__()
        self.snn = self.add_weight(shape=(), initializer="random_normal", trainable=True)

    def call(self, inputs):
        t, x = inputs
        return x - self.snn * t

# Define the neural network architecture
class FpuWaveNet(tf.keras.Model):
    def __init__(self):
        super(FpuWaveNet, self).__init__()
        self.characteristic_layer = CharacteristicLayer()
        self.dense_layers = tf.keras.Sequential([
            tf.keras.layers.Dense(50, activation='relu'),
            tf.keras.layers.Dense(50, activation='relu'),
            tf.keras.layers.Dense(1)
        ])

    def call(self, inputs):
        z = self.characteristic_layer(inputs)
        z = tf.expand_dims(z, -1)  # Ensure z has the right shape for Dense layers
        return self.dense_layers(z)

# Initialize models for U and V
U_nn = FpuWaveNet()
V_nn = FpuWaveNet()

# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Define the loss functions
def loss_function(U_model, V_model, t_points, x_points):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_points)
        U_outputs = U_model([t_points, x_points])
        V_outputs = V_model([t_points, x_points])
        U_x = tape.gradient(U_outputs, x_points)
        U_xx = tape.gradient(U_x, x_points)
        F_V = U_xx + alpha * tape.gradient(U_x * U_x, x_points)

    # Loss for the PDE residuals and boundary conditions
    loss_PDE = tf.reduce_mean(tf.square(V_outputs - F_V))
    loss_boundary = tf.reduce_mean(tf.square(U_outputs)) + tf.reduce_mean(tf.square(V_outputs))

    return loss_PDE + loss_boundary

# Training loop
def train(model_U, model_V, epochs):
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            t_points = tf.random.uniform((100, 1), minval=-a, maxval=a)
            x_points = tf.random.uniform((100, 1), minval=-a, maxval=a)
            loss = loss_function(model_U, model_V, t_points, x_points)
        gradients = tape.gradient(loss, model_U.trainable_variables + model_V.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model_U.trainable_variables + model_V.trainable_variables))
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Start training
train(U_nn, V_nn, epochs)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Generate test data
x_test = np.linspace(-a, a, 400)  # Spatial domain from -a to a
t_test = np.zeros_like(x_test)    # You can vary time to see different time slices
t_test = np.expand_dims(t_test, -1)
x_test = np.expand_dims(x_test, -1)

# Predict using the trained models
U_predictions = U_nn([t_test, x_test]).numpy().flatten()
V_predictions = V_nn([t_test, x_test]).numpy().flatten()

# Visualize the results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(x_test.flatten(), U_predictions, label='U(x)')
plt.title('Profile of U')
plt.xlabel('x')
plt.ylabel('U(x)')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(x_test.flatten(), V_predictions, label='V(x)')
plt.title('Profile of V')
plt.xlabel('x')
plt.ylabel('V(x)')
plt.grid(True)

plt.tight_layout()
plt.show()

# Check residuals
# def compute_residuals(U, V, x):
#     with tf.GradientTape(persistent=True) as tape:
#         tape.watch(x)
#         U_x = tape.gradient(U, x)
#         U_xx = tape.gradient(U_x, x)
#         F_V = U_xx + alpha * tape.gradient(U_x * U_x, x)
#     V_pred = V
#     residual = tf.reduce_mean(tf.square(V_pred - F_V))
#     return residual.numpy()

# residuals = compute_residuals(tf.convert_to_tensor(U_predictions.reshape(-1, 1), dtype=tf.float32),
#                               tf.convert_to_tensor(V_predictions.reshape(-1, 1), dtype=tf.float32),
#                               tf.convert_to_tensor(x_test, dtype=tf.float32))

# print(f"Mean residual error: {residuals}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import tensorflow as tf

# Generate space and time points for prediction
x_values = np.linspace(-a, a, 400)  # spatial domain
time_values = np.linspace(0, 10, 100)  # time domain for the animation
x_grid, t_grid = np.meshgrid(x_values, time_values)

# Prepare data for prediction in correct shape
x_grid_flat = x_grid.flatten()
t_grid_flat = t_grid.flatten()
test_input = [np.expand_dims(t_grid_flat, -1), np.expand_dims(x_grid_flat, -1)]

# Predict using the trained model
U_predictions = U_nn(test_input).numpy().reshape(t_grid.shape)

# Create the animation
fig, ax = plt.subplots()
line, = ax.plot(x_values, U_predictions[0, :], color='b', lw=2)

def update(frame):
    line.set_ydata(U_predictions[frame, :])  # update the data
    return line,

ani = FuncAnimation(fig, update, frames=len(time_values), blit=True)
ax.set_xlim([x_values.min(), x_values.max()])
ax.set_ylim([U_predictions.min(), U_predictions.max()])
ax.set_xlabel('x')
ax.set_ylabel('U(x, t)')
ax.set_title('Traveling Wave Solution Over Time')

# Save the animation
ani.save('traveling_wave.mp4', writer='ffmpeg', fps=15)

plt.show()
